# Nuclear vs. Renewables Generation
### A comparative analysis on how policies and regulations affect energy generation using EIA's API

**Here you will see:**
- Annual electricity generation (thousand MWh) from **Nuclear** and combined **Renewables** (Wind, Solar, Hydro)
- Three compeling case studies using **New York** as our base state, a state leading in Nuclear power generation such as **Illinois** and a state leading in Renewable power generation such as **Iowa**


Setup & Imports:

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

API_KEY = "" # Your key here
states = ["NY", "IL", "IA"]

pd.set_option('display.max_columns', 10)
pd.set_option('display.precision', 2)

## Data retrieval

Let's call the **EIA API** to fetch **annual** generation data for each state, filtered to the Electric Power Sector.

In [ ]:
# Endpoint for EIA electricity operations data
base_url = "https://api.eia.gov/v2/electricity/electric-power-operational-data/data/"

# Container for each state's raw DataFrame
state_dfs = {}

for st in states:
  params = {
      "api_key": API_KEY,
      "frequency": "annual",
      "data[0]": "generation",
      "facets[sectorid][]": "98",
      "facets[location][]": st
  }
  resp = requests.get(base_url, params=params)
  resp.raise_for_status()
  records = resp.json()['response']['data']
  state_dfs[st] = pd.DataFrame(records)

print("Data fetched for states:", list(state_dfs.keys()))

## Inspecting the Raw Data

Let's take a look at New York's raw table to understand its structure.

### We'll be takign a look at the folowing key columns:

- **period** (Year)
- **location** (State code)
- **fueltypeid** & **fueltypedescription** (e.g., Nuclear, Wind, Solar, Hydro)
- **generation** (Value)
- **generation-units** (typically *thousand megawatt-hours*)

In [ ]:
state_dfs["NY"].head(5)

## Cleaning and Preparing the data

To achieve this we will:
1. **Combine** all states into one DataFrame
2. **Select** only Year, State, FuelCode, Fuel, Generation
3. **Filter** to Nuclear (NUC) and Renewables (WND, SUN, HYC)
4. **Fill** missing values -> 0
5. **Aggregate** Renewables = Wind + Solar + Hydro
6. **Merge** Nuclear and Renewables
7. **Pivot** so each year has 'NY_Nuclear', 'NY_Renewables', etc.

In [ ]:
# 1. Combine:
df = pd.concat(state_dfs.values(), ignore_index=True)

# 2. Select and rename
df = df[["period", "location", "fueltypeid", "fuelTypeDescription", "generation"]].rename(
    columns={
        "period": "Year",
        "location": "State",
        "fueltypeid": "FuelCode",
        "fuelTypeDescription": "Fuel"
    }
)
df['Year'] = df['Year'].astype(int)

# 3. Filter fuels
df = df[df['FuelCode'].isin(['NUC', 'WND', 'SUN', 'HYC'])]

# 4. Fill missing -> 0
df['generation'] = pd.to_numeric(df['generation'], errors='coerce').fillna(0.0)

# 5. Aggregate Renewables
renew = (
    df[df['FuelCode'].isin(['WND', 'SUN', 'HYC'])]
    .groupby(['Year', 'State'], as_index=False)['generation']
    .sum()
    .rename(columns={'generation':'Renewables'})
)

# Nuclear
nuc = (
    df[df['FuelCode'] == 'NUC']
    .loc[:, ['Year', 'State', 'generation']]
    .rename(columns={'generation':'Nuclear'})
)

# 6. Merge
merged = pd.merge(nuc, renew, on=['Year', 'State'], how='outer').fillna(0.0)

# 7. Pivot
pivot = (
    merged.pivot(index='Year', columns='State', values=['Nuclear', 'Renewables'])
)
pivot.columns = [f"{state}_{src}" for src, state in pivot.columns]
pivot = pivot.reset_index().sort_values('Year')

## Final Table Preview

Below are the last 10 years of generation (thousand MWh) for each state and source:

In [ ]:
pivot.tail(10)

## Visualizations

Let's plot two lines per state:
- **Nuclear** (solid blue)
- **Renewables** (dashed green)

Each chart will show trends from ~1990 to the latest year.

In [ ]:
# Function to plot one state
def plot_state(state, df):
  yrs = df['Year']
  nuc = df[f"{state}_Nuclear"]
  ren = df[f"{state}_Renewables"]

  plt.figure(figsize=(6, 4))
  plt.plot(yrs, nuc, label='Nuclear', linewidth=2)
  plt.plot(yrs, ren, '--', label='Renewables', linewidth = 2)
  plt.title(f"{state} - Nuclear vs Renewables Generation")
  plt.xlabel("Year")
  plt.ylabel("Generation (thousand MWh)")
  plt.legend()
  plt.grid(alpha=0.3)
  plt.show()


plot_state("NY", pivot)



## New York (NY)
We can see a dramatic shift in New York's energy generation after 2019. Nuclear generation remained between 40,000 and 45,000 thousand MWh but dropped by about 40% between 2019 and 2021 after the closure of the **Indian Point** reactors. After 2020 we see renewables start to overtake nuclear generation. An enhanced **Clean Energy Standard** and policies like the **Climate Leadership and Community Protection Act (2019)** mandated that New York must reach 70% renewable electricity generation by 2030, and 100% carbon free power by 2040.






In [ ]:
plot_state("IL", pivot)

# Illinois (IL)
Illinois has consistently ranked as the nation's top nuclear energy producer. Looking at the chart, we see how nuclear generation has remained consistently above 90,000 thousand MWh for over two decades. Key legislation like the **Future Energy Jobs Act (2016)** and the **Climate and Equitable Jobs Act (2021)** provided subsidies and zero-emission credits to keep nuclear plants economically viable.

While there's a lot more focus on nuclear over renewable generation, Illinois has been making steady progress expanding its renewable energy mix. Around the 2000s renewable generation is near zero and grows to over 25,000 thousand MWh by 2023. This growth is expected to continue due to the state's **Renewable Portfolio Standard** now targeting 40% renewable electricity by 2030.

In [ ]:
plot_state("IA", pivot)

# Iowa (IA)
Iowa is taking a different approach to decarbonization by focusing on renewable energy generation (mostly wind). The state's only nuclear plant generated 4,000 to 6,000 thousand MWh until its closure in 2020, after which nuclear output drops to zero. In contrast, wind generation soared from under 2,000 thousand Mwh in the 2000s to over 45,000 by 2023. This can be attributed to Iowa's early **Renewable Portfolio Standard (1999)**, its **Wind Energy Production Tax Credit**, and the state's abundant prairie wind resources.

Iowa's curve demonstrates that even without nuclear, consistent policy incentives and natural resource advantages can drive rapid renewable adoption.